In [ ]:
```xml
<!-- filepath: c:\Users\jimbet\Dropbox\Teaching\LLM\LLMsInFinance\src\day4\practical-session\03-building-mortgage-rate-estimator.ipynb -->
<VSCode.Cell id="a1b2c3d4" language="markdown">
# Building a Mortgage Rate Estimator with Flask and LLM API

In this notebook, we will build a practical web application that estimates personalized mortgage rates based on client profiles. We'll implement a Flask-based API that leverages LLMs to generate personalized mortgage rate assessments.

## 1. Setup and Dependencies
</VSCode.Cell>
<VSCode.Cell id="e5f6g7h8" language="python">
# Install required packages
!pip install flask openai pandas numpy scikit-learn flask-cors python-dotenv

# Import libraries
import os
import pandas as pd
import numpy as np
from flask import Flask, request, jsonify
from flask_cors import CORS
import json
import time
from openai import OpenAI
from dotenv import load_dotenv
import random
from sklearn.ensemble import RandomForestRegressor
import pickle
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)
</VSCode.Cell>
<VSCode.Cell id="i9j0k1l2" language="markdown">
## 2. Creating a Synthetic Mortgage Dataset

First, we'll create a synthetic dataset of mortgage applicants and rates that we can use to train a baseline model and compare with our LLM-based approach.
</VSCode.Cell>
<VSCode.Cell id="m3n4o5p6" language="python">
def generate_synthetic_mortgage_data(n_samples=1000):
    """
    Generate synthetic mortgage application data
    """
    data = []
    
    # Define parameter ranges
    credit_scores = np.random.randint(550, 850, n_samples)
    loan_amounts = np.random.randint(100000, 1500000, n_samples)
    down_payments = np.random.uniform(0.05, 0.5, n_samples)
    loan_terms = np.random.choice([15, 30], n_samples)
    property_types = np.random.choice(['Single Family', 'Condo', 'Townhouse', 'Multi-Family'], n_samples)
    occupancy_types = np.random.choice(['Primary Residence', 'Second Home', 'Investment'], n_samples)
    debt_to_income = np.random.uniform(0.1, 0.6, n_samples)
    employment_years = np.random.uniform(0, 35, n_samples)
    incomes = np.random.uniform(40000, 500000, n_samples)
    property_locations = np.random.choice([
        'Urban - High Cost', 'Urban - Medium Cost', 'Urban - Low Cost',
        'Suburban - High Cost', 'Suburban - Medium Cost', 'Suburban - Low Cost',
        'Rural - High Cost', 'Rural - Medium Cost', 'Rural - Low Cost'
    ], n_samples)
    
    # Base mortgage rate (e.g., 30-year fixed in 2023)
    base_rate = 0.0675
    
    for i in range(n_samples):
        # Calculate down payment amount
        down_payment_amount = loan_amounts[i] * down_payments[i]
        loan_to_value = 1 - down_payments[i]
        
        # Apply adjustments based on different factors
        credit_adj = (750 - credit_scores[i]) * 0.0001  # Higher credit score -> lower rate
        ltv_adj = (loan_to_value - 0.8) * 0.005 if loan_to_value > 0.8 else 0  # Higher LTV -> higher rate
        term_adj = -0.005 if loan_terms[i] == 15 else 0  # 15-year term -> lower rate
        
        # Property type adjustment
        if property_types[i] == 'Single Family':
            property_adj = 0
        elif property_types[i] == 'Condo':
            property_adj = 0.0025
        elif property_types[i] == 'Townhouse':
            property_adj = 0.001
        else:  # Multi-Family
            property_adj = 0.003
        
        # Occupancy type adjustment
        if occupancy_types[i] == 'Primary Residence':
            occupancy_adj = 0
        elif occupancy_types[i] == 'Second Home':
            occupancy_adj = 0.005
        else:  # Investment
            occupancy_adj = 0.01
        
        # DTI adjustment
        dti_adj = max(0, (debt_to_income[i] - 0.36) * 0.02)
        
        # Calculate mortgage rate with some random noise
        mortgage_rate = base_rate + credit_adj + ltv_adj + term_adj + property_adj + occupancy_adj + dti_adj
        mortgage_rate += np.random.normal(0, 0.001)  # Add some noise
        mortgage_rate = max(mortgage_rate, base_rate - 0.01)  # Set a floor
        
        # Create record
        data.append({
            'credit_score': int(credit_scores[i]),
            'loan_amount': int(loan_amounts[i]),
            'down_payment_percent': round(down_payments[i] * 100, 1),
            'down_payment_amount': int(down_payment_amount),
            'loan_to_value': round(loan_to_value * 100, 1),
            'loan_term': int(loan_terms[i]),
            'property_type': property_types[i],
            'occupancy_type': occupancy_types[i],
            'debt_to_income_ratio': round(debt_to_income[i] * 100, 1),
            'employment_years': round(employment_years[i], 1),
            'annual_income': int(incomes[i]),
            'property_location': property_locations[i],
            'mortgage_rate': round(mortgage_rate * 100, 3)  # Convert to percentage
        })
    
    return pd.DataFrame(data)

# Generate mortgage data
mortgage_df = generate_synthetic_mortgage_data(1500)
mortgage_df.head()
</VSCode.Cell>
<VSCode.Cell id="q7r8s9t0" language="markdown">
Let's visualize some key relationships in our synthetic mortgage data.
</VSCode.Cell>
<VSCode.Cell id="u1v2w3x4" language="python">
import matplotlib.pyplot as plt
import seaborn as sns

# Set up the figure
plt.figure(figsize=(16, 12))

# Plot 1: Credit Score vs Mortgage Rate
plt.subplot(2, 2, 1)
plt.scatter(mortgage_df['credit_score'], mortgage_df['mortgage_rate'], alpha=0.5)
plt.title('Credit Score vs Mortgage Rate')
plt.xlabel('Credit Score')
plt.ylabel('Mortgage Rate (%)')
plt.grid(True, alpha=0.3)

# Plot 2: Loan-to-Value vs Mortgage Rate
plt.subplot(2, 2, 2)
plt.scatter(mortgage_df['loan_to_value'], mortgage_df['mortgage_rate'], alpha=0.5)
plt.title('Loan-to-Value vs Mortgage Rate')
plt.xlabel('Loan-to-Value (%)')
plt.ylabel('Mortgage Rate (%)')
plt.grid(True, alpha=0.3)

# Plot 3: Mortgage Rate by Property Type
plt.subplot(2, 2, 3)
sns.boxplot(x='property_type', y='mortgage_rate', data=mortgage_df)
plt.title('Mortgage Rate by Property Type')
plt.xlabel('Property Type')
plt.ylabel('Mortgage Rate (%)')
plt.xticks(rotation=45)

# Plot 4: Mortgage Rate by Occupancy Type
plt.subplot(2, 2, 4)
sns.boxplot(x='occupancy_type', y='mortgage_rate', data=mortgage_df)
plt.title('Mortgage Rate by Occupancy Type')
plt.xlabel('Occupancy Type')
plt.ylabel('Mortgage Rate (%)')

plt.tight_layout()
plt.show()

# Create a correlation heatmap for numerical features
plt.figure(figsize=(10, 8))
numerical_cols = ['credit_score', 'loan_amount', 'down_payment_percent', 'loan_to_value', 
                 'loan_term', 'debt_to_income_ratio', 'employment_years', 'annual_income', 'mortgage_rate']
correlation = mortgage_df[numerical_cols].corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix of Mortgage Features')
plt.tight_layout()
plt.show()
</VSCode.Cell>
<VSCode.Cell id="y5z6a7b8" language="markdown">
## 3. Build a Baseline Machine Learning Model

Let's create a baseline Random Forest model to predict mortgage rates based on our synthetic data.
</VSCode.Cell>
<VSCode.Cell id="c9d0e1f2" language="python">
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Split data into features and target
X = mortgage_df.drop('mortgage_rate', axis=1)
y = mortgage_df['mortgage_rate']

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define categorical and numerical features
categorical_features = ['property_type', 'occupancy_type', 'property_location']
numerical_features = ['credit_score', 'loan_amount', 'down_payment_percent', 'loan_to_value', 
                      'loan_term', 'debt_to_income_ratio', 'employment_years', 'annual_income']

# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Create and train the model
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Model Performance:")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")
print(f"R-squared: {r2:.4f}")

# Save the model
with open('mortgage_rate_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Feature importance
feature_names = (numerical_features + 
                 list(model.named_steps['preprocessor']
                     .transformers_[1][1]
                     .get_feature_names_out(categorical_features)))

importances = model.named_steps['regressor'].feature_importances_

# Create a DataFrame for easier visualization
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values('Importance', ascending=False)

# Plot feature importance
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=importance_df.head(15))
plt.title('Top 15 Features by Importance')
plt.tight_layout()
plt.show()
</VSCode.Cell>
<VSCode.Cell id="g3h4i5j6" language="markdown">
## 4. Build the LLM-Based Mortgage Rate Estimator

Now we'll build an LLM-based approach to estimating mortgage rates that can incorporate more contextual information.
</VSCode.Cell>
<VSCode.Cell id="k7l8m9n0" language="python">
def llm_mortgage_rate_estimate(client_profile, model="gpt-4-turbo", market_context=None):
    """
    Use an LLM to estimate a personalized mortgage rate and provide explanation
    
    Args:
        client_profile: Dictionary with client information
        model: LLM model to use
        market_context: Optional current market conditions to consider
    
    Returns:
        Dictionary with estimated rate and explanation
    """
    # Initialize OpenAI client
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "your-api-key-here"))
    
    # Set default market context if none provided
    if market_context is None:
        market_context = {
            "fed_funds_rate": 5.5,
            "ten_year_treasury": 4.2,
            "average_30yr_fixed": 6.75,
            "average_15yr_fixed": 6.1,
            "housing_market_trend": "Stable with modest price growth",
            "economic_outlook": "Moderate growth with inflation concerns"
        }
    
    # Format the prompt
    prompt = f"""You are an expert mortgage loan underwriter and financial analyst. Estimate an appropriate mortgage interest rate for the following client profile, based on current market conditions.

CLIENT PROFILE:
- Credit Score: {client_profile.get('credit_score')}
- Loan Amount: ${client_profile.get('loan_amount'):,}
- Down Payment: {client_profile.get('down_payment_percent')}%
- Loan-to-Value Ratio: {client_profile.get('loan_to_value')}%
- Loan Term: {client_profile.get('loan_term')} years
- Property Type: {client_profile.get('property_type')}
- Occupancy Type: {client_profile.get('occupancy_type')}
- Debt-to-Income Ratio: {client_profile.get('debt_to_income_ratio')}%
- Years of Employment: {client_profile.get('employment_years')}
- Annual Income: ${client_profile.get('annual_income'):,}
- Property Location: {client_profile.get('property_location')}

CURRENT MARKET CONDITIONS:
- Federal Funds Rate: {market_context.get('fed_funds_rate')}%
- 10-Year Treasury Yield: {market_context.get('ten_year_treasury')}%
- Average 30-Year Fixed Rate: {market_context.get('average_30yr_fixed')}%
- Average 15-Year Fixed Rate: {market_context.get('average_15yr_fixed')}%
- Housing Market Trend: {market_context.get('housing_market_trend')}
- Economic Outlook: {market_context.get('economic_outlook')}

Based on the client profile and current market conditions, please:
1. Estimate the appropriate mortgage interest rate for this client
2. Explain the factors that most influenced your rate determination
3. Indicate if there are any specific risks or concerns with this application
4. Suggest potential ways the client could improve their rate

Format your response as a JSON object with the following structure:
{{
    "estimated_rate": rate_as_percentage,
    "rate_components": {{
        "base_rate": base_rate_percentage,
        "credit_adjustment": credit_adjustment,
        "ltv_adjustment": ltv_adjustment,
        "occupancy_adjustment": occupancy_adjustment,
        "property_type_adjustment": property_type_adjustment,
        "dti_adjustment": dti_adjustment,
        "term_adjustment": term_adjustment,
        "location_adjustment": location_adjustment
    }},
    "key_factors": [
        "Description of factor 1 that influenced the rate",
        "Description of factor 2 that influenced the rate",
        ...
    ],
    "risk_assessment": "Overall risk assessment",
    "improvement_suggestions": [
        "Suggestion 1 to improve rate",
        "Suggestion 2 to improve rate",
        ...
    ]
}}
"""

    # Call the LLM API
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are an expert mortgage underwriter with deep knowledge of the mortgage industry and risk assessment."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2,  # Low temperature for consistency
            response_format={"type": "json_object"}  # Request JSON output
        )
        
        # Parse the JSON response
        result = json.loads(response.choices[0].message.content)
        
        # Add metadata
        result["model_used"] = model
        result["client_profile"] = client_profile
        result["market_context"] = market_context
        
        return result
    
    except Exception as e:
        print(f"Error estimating mortgage rate: {e}")
        return {"error": str(e)}
</VSCode.Cell>
<VSCode.Cell id="o1p2q3r4" language="markdown">
Let's test our LLM-based estimator with a few sample client profiles.
</VSCode.Cell>
<VSCode.Cell id="s5t6u7v8" language="python">
# Sample client profiles
sample_clients = [
    # Strong profile
    {
        'credit_score': 780,
        'loan_amount': 400000,
        'down_payment_percent': 20.0,
        'loan_to_value': 80.0,
        'loan_term': 30,
        'property_type': 'Single Family',
        'occupancy_type': 'Primary Residence',
        'debt_to_income_ratio': 28.0,
        'employment_years': 10.5,
        'annual_income': 150000,
        'property_location': 'Suburban - Medium Cost'
    },
    
    # Moderate risk profile
    {
        'credit_score': 680,
        'loan_amount': 350000,
        'down_payment_percent': 10.0,
        'loan_to_value': 90.0,
        'loan_term': 30,
        'property_type': 'Condo',
        'occupancy_type': 'Primary Residence',
        'debt_to_income_ratio': 42.0,
        'employment_years': 3.5,
        'annual_income': 85000,
        'property_location': 'Urban - High Cost'
    },
    
    # Higher risk profile
    {
        'credit_score': 620,
        'loan_amount': 275000,
        'down_payment_percent': 5.0,
        'loan_to_value': 95.0,
        'loan_term': 30,
        'property_type': 'Townhouse',
        'occupancy_type': 'Investment',
        'debt_to_income_ratio': 48.0,
        'employment_years': 1.2,
        'annual_income': 70000,
        'property_location': 'Urban - Low Cost'
    }
]

# Test the LLM estimator with each profile
llm_results = []

for i, client in enumerate(sample_clients):
    print(f"Processing client profile {i+1}...")
    result = llm_mortgage_rate_estimate(client)
    llm_results.append(result)
    
    # Compare with baseline model prediction
    baseline_pred = model.predict([pd.Series(client)])[0]
    
    print(f"\nClient Profile {i+1} Results:")
    print(f"LLM Estimated Rate: {result['estimated_rate']}%")
    print(f"Baseline Model Rate: {baseline_pred:.3f}%")
    print(f"Difference: {abs(result['estimated_rate'] - baseline_pred):.3f}%\n")
    
    # Display key factors
    print("Key Factors:")
    for factor in result['key_factors']:
        print(f"- {factor}")
    
    print("\nImprovement Suggestions:")
    for suggestion in result['improvement_suggestions']:
        print(f"- {suggestion}")
    
    print("\n" + "-"*50 + "\n")
</VSCode.Cell>
<VSCode.Cell id="w9x0y1z2" language="markdown">
## 5. Create a Flask API for the Mortgage Rate Estimator

Now let's create a Flask API that can serve our mortgage rate estimator as a web service.
</VSCode.Cell>
<VSCode.Cell id="a3b4c5d6" language="python">
# Define the Flask application
app = Flask(__name__)
CORS(app)  # Enable Cross-Origin Resource Sharing

# Load the pre-trained model
try:
    with open('mortgage_rate_model.pkl', 'rb') as f:
        baseline_model = pickle.load(f)
except:
    # If we don't have the model saved, recreate it
    baseline_model = model

@app.route('/api/estimate_rate', methods=['POST'])
def estimate_rate():
    """
    API endpoint for estimating mortgage rates
    """
    try:
        # Get data from request
        data = request.json
        client_profile = data.get('client_profile')
        estimate_method = data.get('method', 'both')  # 'llm', 'ml', or 'both'
        market_context = data.get('market_context')
        
        # Validate required fields
        required_fields = ['credit_score', 'loan_amount', 'down_payment_percent', 
                         'loan_to_value', 'loan_term', 'property_type', 
                         'occupancy_type', 'debt_to_income_ratio']
        
        for field in required_fields:
            if field not in client_profile:
                return jsonify({
                    'error': f'Missing required field: {field}'
                }), 400
        
        response = {}
        
        # Get ML model estimate if requested
        if estimate_method in ['ml', 'both']:
            try:
                # Convert client profile to DataFrame row
                client_df = pd.DataFrame([client_profile])
                
                # Make prediction
                ml_rate = float(baseline_model.predict(client_df)[0])
                
                response['ml_estimate'] = {
                    'rate': round(ml_rate, 3),
                    'model_type': 'Random Forest Regressor'
                }
            except Exception as e:
                response['ml_estimate'] = {
                    'error': f"ML estimation failed: {str(e)}"
                }
        
        # Get LLM estimate if requested
        if estimate_method in ['llm', 'both']:
            try:
                llm_result = llm_mortgage_rate_estimate(client_profile, market_context=market_context)
                response['llm_estimate'] = llm_result
            except Exception as e:
                response['llm_estimate'] = {
                    'error': f"LLM estimation failed: {str(e)}"
                }
        
        return jsonify(response)
    
    except Exception as e:
        return jsonify({
            'error': f'An error occurred: {str(e)}'
        }), 500

@app.route('/api/health', methods=['GET'])
def health_check():
    """
    Health check endpoint
    """
    return jsonify({
        'status': 'healthy',
        'message': 'Mortgage Rate Estimator API is running'
    })

# Example function to start the app (for running directly in this notebook)
def run_flask_app():
    app.run(host='0.0.0.0', port=5000, debug=True)

# Note: In a production environment, you would run this with a proper WSGI server
# The following code is just for demonstration purposes

# Create a simple HTML page to test our API
html_code = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Mortgage Rate Estimator</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha1/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        .rate-result {
            font-size: 24px;
            font-weight: bold;
            margin: 20px 0;
        }
        .rate-component {
            margin-bottom: 5px;
        }
        .loading {
            display: none;
        }
    </style>
</head>
<body>
    <div class="container mt-5">
        <h1 class="mb-4">Mortgage Rate Estimator</h1>
        
        <div class="row">
            <div class="col-md-6">
                <div class="card">
                    <div class="card-header">
                        <h5>Client Profile</h5>
                    </div>
                    <div class="card-body">
                        <form id="rateForm">
                            <div class="mb-3">
                                <label for="creditScore" class="form-label">Credit Score</label>
                                <input type="number" class="form-control" id="creditScore" min="300" max="850" value="720">
                            </div>
                            
                            <div class="mb-3">
                                <label for="loanAmount" class="form-label">Loan Amount ($)</label>
                                <input type="number" class="form-control" id="loanAmount" min="50000" value="350000">
                            </div>
                            
                            <div class="mb-3">
                                <label for="downPayment" class="form-label">Down Payment (%)</label>
                                <input type="number" class="form-control" id="downPayment" min="3" max="50" value="20">
                            </div>
                            
                            <div class="mb-3">
                                <label for="loanTerm" class="form-label">Loan Term (Years)</label>
                                <select class="form-control" id="loanTerm">
                                    <option value="15">15 Years</option>
                                    <option value="30" selected>30 Years</option>
                                </select>
                            </div>
                            
                            <div class="mb-3">
                                <label for="propertyType" class="form-label">Property Type</label>
                                <select class="form-control" id="propertyType">
                                    <option value="Single Family" selected>Single Family</option>
                                    <option value="Condo">Condo</option>
                                    <option value="Townhouse">Townhouse</option>
                                    <option value="Multi-Family">Multi-Family</option>
                                </select>
                            </div>
                            
                            <div class="mb-3">
                                <label for="occupancyType" class="form-label">Occupancy Type</label>
                                <select class="form-control" id="occupancyType">
                                    <option value="Primary Residence" selected>Primary Residence</option>
                                    <option value="Second Home">Second Home</option>
                                    <option value="Investment">Investment</option>
                                </select>
                            </div>
                            
                            <div class="mb-3">
                                <label for="dtiRatio" class="form-label">Debt-to-Income Ratio (%)</label>
                                <input type="number" class="form-control" id="dtiRatio" min="10" max="65" value="36">
                            </div>
                            
                            <div class="mb-3">
                                <label for="employmentYears" class="form-label">Years of Employment</label>
                                <input type="number" class="form-control" id="employmentYears" min="0" step="0.5" value="5">
                            </div>
                            
                            <div class="mb-3">
                                <label for="annualIncome" class="form-label">Annual Income ($)</label>
                                <input type="number" class="form-control" id="annualIncome" min="20000" value="90000">
                            </div>
                            
                            <div class="mb-3">
                                <label for="propertyLocation" class="form-label">Property Location</label>
                                <select class="form-control" id="propertyLocation">
                                    <option value="Urban - High Cost">Urban - High Cost</option>
                                    <option value="Urban - Medium Cost">Urban - Medium Cost</option>
                                    <option value="Urban - Low Cost">Urban - Low Cost</option>
                                    <option value="Suburban - High Cost">Suburban - High Cost</option>
                                    <option value="Suburban - Medium Cost" selected>Suburban - Medium Cost</option>
                                    <option value="Suburban - Low Cost">Suburban - Low Cost</option>
                                    <option value="Rural - High Cost">Rural - High Cost</option>
                                    <option value="Rural - Medium Cost">Rural - Medium Cost</option>
                                    <option value="Rural - Low Cost">Rural - Low Cost</option>
                                </select>
                            </div>
                            
                            <div class="mb-3">
                                <label for="estimateMethod" class="form-label">Estimation Method</label>
                                <select class="form-control" id="estimateMethod">
                                    <option value="both" selected>Both (ML + LLM)</option>
                                    <option value="llm">LLM Only</option>
                                    <option value="ml">ML Model Only</option>
                                </select>
                            </div>
                            
                            <button type="submit" class="btn btn-primary">Estimate Rate</button>
                        </form>
                    </div>
                </div>
            </div>
            
            <div class="col-md-6">
                <div class="card">
                    <div class="card-header">
                        <h5>Rate Estimate Results</h5>
                    </div>
                    <div class="card-body">
                        <div class="loading text-center">
                            <div class="spinner-border text-primary" role="status">
                                <span class="visually-hidden">Loading...</span>
                            </div>
                            <p class="mt-2">Calculating rate estimate...</p>
                        </div>
                        
                        <div id="results" class="d-none">
                            <div id="mlResult" class="mb-4 d-none">
                                <h4>Machine Learning Estimate</h4>
                                <div class="rate-result" id="mlRate"></div>
                            </div>
                            
                            <div id="llmResult" class="d-none">
                                <h4>LLM-Based Estimate</h4>
                                <div class="rate-result" id="llmRate"></div>
                                
                                <h5 class="mt-4">Rate Components</h5>
                                <div id="rateComponents"></div>
                                
                                <h5 class="mt-4">Key Factors</h5>
                                <ul id="keyFactors" class="list-group"></ul>
                                
                                <h5 class="mt-4">Risk Assessment</h5>
                                <div id="riskAssessment" class="alert alert-secondary"></div>
                                
                                <h5 class="mt-4">Improvement Suggestions</h5>
                                <ul id="improvementSuggestions" class="list-group"></ul>
                            </div>
                        </div>
                    </div>
                </div>
            </div>
        </div>
    </div>
    
    <script>
        document.getElementById('rateForm').addEventListener('submit', async function(e) {
            e.preventDefault();
            
            // Show loading indicator
            document.querySelector('.loading').style.display = 'block';
            document.getElementById('results').classList.add('d-none');
            
            // Get form values
            const creditScore = parseInt(document.getElementById('creditScore').value);
            const loanAmount = parseInt(document.getElementById('loanAmount').value);
            const downPayment = parseFloat(document.getElementById('downPayment').value);
            const loanTerm = parseInt(document.getElementById('loanTerm').value);
            const propertyType = document.getElementById('propertyType').value;
            const occupancyType = document.getElementById('occupancyType').value;
            const dtiRatio = parseFloat(document.getElementById('dtiRatio').value);
            const employmentYears = parseFloat(document.getElementById('employmentYears').value);
            const annualIncome = parseInt(document.getElementById('annualIncome').value);
            const propertyLocation = document.getElementById('propertyLocation').value;
            const estimateMethod = document.getElementById('estimateMethod').value;
            
            // Calculate loan-to-value
            const loanToValue = 100 - downPayment;
            
            // Create client profile
            const clientProfile = {
                credit_score: creditScore,
                loan_amount: loanAmount,
                down_payment_percent: downPayment,
                loan_to_value: loanToValue,
                loan_term: loanTerm,
                property_type: propertyType,
                occupancy_type: occupancyType,
                debt_to_income_ratio: dtiRatio,
                employment_years: employmentYears,
                annual_income: annualIncome,
                property_location: propertyLocation
            };
            
            // Create market context
            const marketContext = {
                fed_funds_rate: 5.5,
                ten_year_treasury: 4.2,
                average_30yr_fixed: 6.75,
                average_15yr_fixed: 6.1,
                housing_market_trend: "Stable with modest price growth",
                economic_outlook: "Moderate growth with inflation concerns"
            };
            
            try {
                // Call API
                const response = await fetch('/api/estimate_rate', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json'
                    },
                    body: JSON.stringify({
                        client_profile: clientProfile,
                        method: estimateMethod,
                        market_context: marketContext
                    })
                });
                
                const data = await response.json();
                
                // Hide loading indicator
                document.querySelector('.loading').style.display = 'none';
                document.getElementById('results').classList.remove('d-none');
                
                // Display ML results if available
                if (data.ml_estimate) {
                    document.getElementById('mlResult').classList.remove('d-none');
                    document.getElementById('mlRate').textContent = `${data.ml_estimate.rate}%`;
                } else {
                    document.getElementById('mlResult').classList.add('d-none');
                }
                
                // Display LLM results if available
                if (data.llm_estimate && !data.llm_estimate.error) {
                    document.getElementById('llmResult').classList.remove('d-none');
                    document.getElementById('llmRate').textContent = `${data.llm_estimate.estimated_rate}%`;
                    
                    // Display rate components
                    const componentsDiv = document.getElementById('rateComponents');
                    componentsDiv.innerHTML = '';
                    for (const [component, value] of Object.entries(data.llm_estimate.rate_components)) {
                        const formattedComponent = component.replace(/_/g, ' ').replace(/\b\w/g, l => l.toUpperCase());
                        componentsDiv.innerHTML += `<div class="rate-component">${formattedComponent}: ${value}%</div>`;
                    }
                    
                    // Display key factors
                    const factorsUl = document.getElementById('keyFactors');
                    factorsUl.innerHTML = '';
                    data.llm_estimate.key_factors.forEach(factor => {
                        factorsUl.innerHTML += `<li class="list-group-item">${factor}</li>`;
                    });
                    
                    // Display risk assessment
                    document.getElementById('riskAssessment').textContent = data.llm_estimate.risk_assessment;
                    
                    // Display improvement suggestions
                    const suggestionsUl = document.getElementById('improvementSuggestions');
                    suggestionsUl.innerHTML = '';
                    data.llm_estimate.improvement_suggestions.forEach(suggestion => {
                        suggestionsUl.innerHTML += `<li class="list-group-item">${suggestion}</li>`;
                    });
                } else {
                    document.getElementById('llmResult').classList.add('d-none');
                }
                
            } catch (error) {
                console.error('Error:', error);
                document.querySelector('.loading').style.display = 'none';
                alert('An error occurred while estimating the rate. Please try again.');
            }
        });
    </script>
</body>
</html>
"""

# Save the HTML to a file for demonstration
with open('mortgage_rate_estimator.html', 'w') as f:
    f.write(html_code)

print("HTML file created: mortgage_rate_estimator.html")
print("To run the Flask app, execute the following in a Python script:")
print("from mortgage_api import app")
print("app.run(host='0.0.0.0', port=5000)")
</VSCode.Cell>
<VSCode.Cell id="e7f8g9h0" language="markdown">
Let's create a Python script file that can be used to run our Flask application.
</VSCode.Cell>
<VSCode.Cell id="i1j2k3l4" language="python">
%%writefile mortgage_api.py
# Import necessary libraries
import os
import pandas as pd
import numpy as np
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import json
import pickle
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Flask app
app = Flask(__name__)
CORS(app)  # Enable Cross-Origin Resource Sharing

# Load the pre-trained model
try:
    with open('mortgage_rate_model.pkl', 'rb') as f:
        baseline_model = pickle.load(f)
    print("Loaded pre-trained model from file")
except:
    print("Could not load model from file - model needs to be trained first")
    baseline_model = None

# Function for LLM-based mortgage rate estimation
def llm_mortgage_rate_estimate(client_profile, model="gpt-4-turbo", market_context=None):
    """
    Use an LLM to estimate a personalized mortgage rate and provide explanation
    
    Args:
        client_profile: Dictionary with client information
        model: LLM model to use
        market_context: Optional current market conditions to consider
    
    Returns:
        Dictionary with estimated rate and explanation
    """
    # Initialize OpenAI client
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "your-api-key-here"))
    
    # Set default market context if none provided
    if market_context is None:
        market_context = {
            "fed_funds_rate": 5.5,
            "ten_year_treasury": 4.2,
            "average_30yr_fixed": 6.75,
            "average_15yr_fixed": 6.1,
            "housing_market_trend": "Stable with modest price growth",
            "economic_outlook": "Moderate growth with inflation concerns"
        }
    
    # Format the prompt
    prompt = f"""You are an expert mortgage loan underwriter and financial analyst. Estimate an appropriate mortgage interest rate for the following client profile, based on current market conditions.

CLIENT PROFILE:
- Credit Score: {client_profile.get('credit_score')}
- Loan Amount: ${client_profile.get('loan_amount'):,}
- Down Payment: {client_profile.get('down_payment_percent')}%
- Loan-to-Value Ratio: {client_profile.get('loan_to_value')}%
- Loan Term: {client_profile.get('loan_term')} years
- Property Type: {client_profile.get('property_type')}
- Occupancy Type: {client_profile.get('occupancy_type')}
- Debt-to-Income Ratio: {client_profile.get('debt_to_income_ratio')}%
- Years of Employment: {client_profile.get('employment_years')}
- Annual Income: ${client_profile.get('annual_income'):,}
- Property Location: {client_profile.get('property_location')}

CURRENT MARKET CONDITIONS:
- Federal Funds Rate: {market_context.get('fed_funds_rate')}%
- 10-Year Treasury Yield: {market_context.get('ten_year_treasury')}%
- Average 30-Year Fixed Rate: {market_context.get('average_30yr_fixed')}%
- Average 15-Year Fixed Rate: {market_context.get('average_15yr_fixed')}%
- Housing Market Trend: {market_context.get('housing_market_trend')}
- Economic Outlook: {market_context.get('economic_outlook')}

Based on the client profile and current market conditions, please:
1. Estimate the appropriate mortgage interest rate for this client
2. Explain the factors that most influenced your rate determination
3. Indicate if there are any specific risks or concerns with this application
4. Suggest potential ways the client could improve their rate

Format your response as a JSON object with the following structure:
{{
    "estimated_rate": rate_as_percentage,
    "rate_components": {{
        "base_rate": base_rate_percentage,
        "credit_adjustment": credit_adjustment,
        "ltv_adjustment": ltv_adjustment,
        "occupancy_adjustment": occupancy_adjustment,
        "property_type_adjustment": property_type_adjustment,
        "dti_adjustment": dti_adjustment,
        "term_adjustment": term_adjustment,
        "location_adjustment": location_adjustment
    }},
    "key_factors": [
        "Description of factor 1 that influenced the rate",
        "Description of factor 2 that influenced the rate",
        ...
    ],
    "risk_assessment": "Overall risk assessment",
    "improvement_suggestions": [
        "Suggestion 1 to improve rate",
        "Suggestion 2 to improve rate",
        ...
    ]
}}
"""

    # Call the LLM API
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are an expert mortgage underwriter with deep knowledge of the mortgage industry and risk assessment."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2,  # Low temperature for consistency
            response_format={"type": "json_object"}  # Request JSON output
        )
        
        # Parse the JSON response
        result = json.loads(response.choices[0].message.content)
        
        # Add metadata
        result["model_used"] = model
        result["client_profile"] = client_profile
        result["market_context"] = market_context
        
        return result
    
    except Exception as e:
        print(f"Error estimating mortgage rate: {e}")
        return {"error": str(e)}

# API endpoint for estimating mortgage rates
@app.route('/api/estimate_rate', methods=['POST'])
def estimate_rate():
    """
    API endpoint for estimating mortgage rates
    """
    try:
        # Get data from request
        data = request.json
        client_profile = data.get('client_profile')
        estimate_method = data.get('method', 'both')  # 'llm', 'ml', or 'both'
        market_context = data.get('market_context')
        
        # Validate required fields
        required_fields = ['credit_score', 'loan_amount', 'down_payment_percent', 
                         'loan_to_value', 'loan_term', 'property_type', 
                         'occupancy_type', 'debt_to_income_ratio']
        
        for field in required_fields:
            if field not in client_profile:
                return jsonify({
                    'error': f'Missing required field: {field}'
                }), 400
        
        response = {}
        
        # Get ML model estimate if requested
        if estimate_method in ['ml', 'both'] and baseline_model is not None:
            try:
                # Convert client profile to DataFrame row
                client_df = pd.DataFrame([client_profile])
                
                # Make prediction
                ml_rate = float(baseline_model.predict(client_df)[0])
                
                response['ml_estimate'] = {
                    'rate': round(ml_rate, 3),
                    'model_type': 'Random Forest Regressor'
                }
            except Exception as e:
                response['ml_estimate'] = {
                    'error': f"ML estimation failed: {str(e)}"
                }
        
        # Get LLM estimate if requested
        if estimate_method in ['llm', 'both']:
            try:
                llm_result = llm_mortgage_rate_estimate(client_profile, market_context=market_context)
                response['llm_estimate'] = llm_result
            except Exception as e:
                response['llm_estimate'] = {
                    'error': f"LLM estimation failed: {str(e)}"
                }
        
        return jsonify(response)
    
    except Exception as e:
        return jsonify({
            'error': f'An error occurred: {str(e)}'
        }), 500

# Health check endpoint
@app.route('/api/health', methods=['GET'])
def health_check():
    """
    Health check endpoint
    """
    return jsonify({
        'status': 'healthy',
        'message': 'Mortgage Rate Estimator API is running'
    })

# Serve the HTML file
@app.route('/', methods=['GET'])
def serve_app():
    return send_from_directory('.', 'mortgage_rate_estimator.html')

# Run the Flask app if executed directly
if __name__ == '__main__':
    port = int(os.environ.get('PORT', 5000))
    app.run(host='0.0.0.0', port=port, debug=True)
</VSCode.Cell>
<VSCode.Cell id="m5n6o7p8" language="markdown">
## 6. Analyzing the Differences Between ML and LLM Approaches

Let's compare the machine learning and LLM approaches to mortgage rate estimation and analyze their strengths and weaknesses.
</VSCode.Cell>
<VSCode.Cell id="q9r0s1t2" language="python">
# Create a function to compare ML and LLM estimates across a range of scenarios
def compare_ml_llm_estimates(test_cases):
    """
    Compare ML and LLM mortgage rate estimates across various test cases
    """
    results = []
    
    for i, case in enumerate(test_cases):
        print(f"Processing test case {i+1}/{len(test_cases)}...")
        
        # Get ML estimate
        ml_estimate = float(model.predict([pd.Series(case)])[0])
        
        # Get LLM estimate
        llm_result = llm_mortgage_rate_estimate(case)
        llm_estimate = llm_result.get('estimated_rate')
        
        # Create result object
        result = {
            'test_case': i + 1,
            'client_profile': case,
            'ml_estimate': ml_estimate,
            'llm_estimate': llm_estimate,
            'difference': abs(ml_estimate - llm_estimate) if llm_estimate is not None else None,
            'key_factors': llm_result.get('key_factors', []),
            'risk_assessment': llm_result.get('risk_assessment', ''),
            'improvement_suggestions': llm_result.get('improvement_suggestions', [])
        }
        
        results.append(result)
    
    return results

# Generate diverse test cases
import itertools

# Define parameter ranges for test cases
test_param_ranges = {
    'credit_score': [620, 680, 740, 800],
    'loan_to_value': [70, 80, 90, 95],
    'occupancy_type': ['Primary Residence', 'Investment'],
    'property_type': ['Single Family', 'Condo'],
    'loan_term': [15, 30],
    'debt_to_income_ratio': [28, 36, 45]
}

# Create test cases from combinations of parameters
test_cases = []

# Use a subset of combinations to keep the number manageable
for credit_score, ltv, occupancy, property_type in itertools.product(
    test_param_ranges['credit_score'],
    test_param_ranges['loan_to_value'],
    test_param_ranges['occupancy_type'],
    test_param_ranges['property_type']
):
    # Fixed values for other parameters
    loan_amount = 350000
    down_payment = 100 - ltv
    
    test_case = {
        'credit_score': credit_score,
        'loan_amount': loan_amount,
        'down_payment_percent': down_payment,
        'loan_to_value': ltv,
        'loan_term': 30,
        'property_type': property_type,
        'occupancy_type': occupancy,
        'debt_to_income_ratio': 36,
        'employment_years': 5,
        'annual_income': 100000,
        'property_location': 'Suburban - Medium Cost'
    }
    
    test_cases.append(test_case)

# Limit to 10 test cases for demonstration
limited_test_cases = test_cases[:10]

# Run comparison (note: this will use API credits for each case)
# comparison_results = compare_ml_llm_estimates(limited_test_cases)

# Instead of running the actual comparison which would use API credits,
# let's generate a simulated comparison for demonstration purposes
def simulate_comparison_results(test_cases):
    """
    Simulate comparison results for demonstration
    """
    simulated_results = []
    
    for i, case in enumerate(test_cases):
        # Base rate around 6.75%
        base_rate = 6.75
        
        # ML estimate: Systematic with some noise
        credit_adj = (750 - case['credit_score']) * 0.0005
        ltv_adj = (case['loan_to_value'] - 80) * 0.003 if case['loan_to_value'] > 80 else 0
        occ_adj = 0.25 if case['occupancy_type'] == 'Investment' else 0
        prop_adj = 0.125 if case['property_type'] == 'Condo' else 0
        
        ml_estimate = base_rate + credit_adj + ltv_adj + occ_adj + prop_adj + np.random.normal(0, 0.05)
        ml_estimate = round(ml_estimate, 3)
        
        # LLM estimate: More context-aware with different factors
        llm_base = 6.7  # Slightly different base assessment
        llm_credit_adj = (750 - case['credit_score']) * 0.0006
        llm_ltv_adj = (case['loan_to_value'] - 75) * 0.004 if case['loan_to_value'] > 75 else 0
        llm_occ_adj = 0.3 if case['occupancy_type'] == 'Investment' else 0
        llm_prop_adj = 0.15 if case['property_type'] == 'Condo' else 0
        
        # LLM might consider more complex interactions
        if case['credit_score'] < 680 and case['loan_to_value'] > 90:
            llm_risk_adj = 0.2  # Higher adjustment for risky combinations
        else:
            llm_risk_adj = 0
            
        llm_estimate = llm_base + llm_credit_adj + llm_ltv_adj + llm_occ_adj + llm_prop_adj + llm_risk_adj
        llm_estimate = round(llm_estimate, 2)
        
        # Generate key factors based on profile
        key_factors = []
        if case['credit_score'] < 700:
            key_factors.append(f"Credit score of {case['credit_score']} is below the excellent range, leading to a rate adjustment")
        
        if case['loan_to_value'] > 80:
            key_factors.append(f"Loan-to-value ratio of {case['loan_to_value']}% exceeds 80%, requiring mortgage insurance and a rate adjustment")
            
        if case['occupancy_type'] == 'Investment':
            key_factors.append("Investment property carries higher risk than a primary residence, resulting in a rate premium")
            
        if case['property_type'] == 'Condo':
            key_factors.append("Condo properties typically receive slightly higher rates due to additional association risks")
        
        # Risk assessment
        if case['credit_score'] >= 740 and case['loan_to_value'] <= 80 and case['occupancy_type'] == 'Primary Residence':
            risk = "Low risk profile. Strong credit score, good down payment, and primary residence status."
        elif case['credit_score'] < 660 or case['loan_to_value'] > 90:
            risk = "Elevated risk profile due to either lower credit score or high loan-to-value ratio."
        else:
            risk = "Moderate risk profile with standard underwriting considerations."
            
        # Improvement suggestions
        improvements = []
        if case['credit_score'] < 740:
            improvements.append("Improve credit score to qualify for better rates")
        
        if case['loan_to_value'] > 80:
            improvements.append("Increase down payment to at least 20% to avoid mortgage insurance and improve rate")
            
        if case['debt_to_income_ratio'] > 36:
            improvements.append("Reduce overall debt to improve debt-to-income ratio")
        
        # Create result
        result = {
            'test_case': i + 1,
            'client_profile': case,
            'ml_estimate': ml_estimate,
            'llm_estimate': llm_estimate,
            'difference': round(abs(ml_estimate - llm_estimate), 3),
            'key_factors': key_factors,
            'risk_assessment': risk,
            'improvement_suggestions': improvements
        }
        
        simulated_results.append(result)
    
    return simulated_results

# Simulate comparison results
comparison_results = simulate_comparison_results(limited_test_cases)

# Analyze the comparison results
comparison_df = pd.DataFrame(comparison_results)

# Calculate summary statistics
mean_diff = comparison_df['difference'].mean()
max_diff = comparison_df['difference'].max()
min_diff = comparison_df['difference'].min()

print(f"Comparison Summary:")
print(f"Average Difference: {mean_diff:.3f}%")
print(f"Maximum Difference: {max_diff:.3f}%")
print(f"Minimum Difference: {min_diff:.3f}%")

# Plot the comparison
plt.figure(figsize=(14, 8))

# Sort by ML estimate for better visualization
comparison_df = comparison_df.sort_values('ml_estimate')

# Create bar chart of estimates
x = np.arange(len(comparison_df))
width = 0.35

plt.bar(x - width/2, comparison_df['ml_estimate'], width, label='ML Estimate')
plt.bar(x + width/2, comparison_df['llm_estimate'], width, label='LLM Estimate')

# Add labels and title
plt.xlabel('Test Case')
plt.ylabel('Mortgage Rate (%)')
plt.title('Comparison of ML vs. LLM Mortgage Rate Estimates')
plt.xticks(x, [f"Case {i}" for i in comparison_df['test_case']])
plt.legend()

# Add grid
plt.grid(axis='y', alpha=0.3)

# Add text labels for each case
for i, row in enumerate(comparison_df.iterrows()):
    _, data = row
    cs = data['client_profile']['credit_score']
    ltv = data['client_profile']['loan_to_value']
    occ = 'Inv' if data['client_profile']['occupancy_type'] == 'Investment' else 'Prim'
    prop = 'Cnd' if data['client_profile']['property_type'] == 'Condo' else 'SFH'
    
    label = f"CS:{cs}\nLTV:{ltv}\n{occ}/{prop}"
    plt.annotate(label, xy=(i, 0.5), xytext=(0, -25),
                textcoords="offset points", ha='center', va='top',
                fontsize=8)

plt.tight_layout()
plt.show()

# Create a table of key differences
from IPython.display import HTML, display

# Extract only relevant columns for display
display_df = comparison_df[['test_case', 'ml_estimate', 'llm_estimate', 'difference']]
display_df = display_df.rename(columns={
    'test_case': 'Case',
    'ml_estimate': 'ML Estimate (%)',
    'llm_estimate': 'LLM Estimate (%)',
    'difference': 'Difference (%)'
})

# Display as a styled table
display(HTML(display_df.to_html(index=False)))

# Display LLM insights for the largest difference case
max_diff_case = comparison_df.loc[comparison_df['difference'].idxmax()]

print(f"\nInsights for Case {max_diff_case['test_case']} (Largest Difference):")
print(f"ML Estimate: {max_diff_case['ml_estimate']}%")
print(f"LLM Estimate: {max_diff_case['llm_estimate']}%")
print(f"Difference: {max_diff_case['difference']}%")

print("\nKey Factors:")
for factor in max_diff_case['key_factors']:
    print(f"- {factor}")

print(f"\nRisk Assessment: {max_diff_case['risk_assessment']}")

print("\nImprovement Suggestions:")
for suggestion in max_diff_case['improvement_suggestions']:
    print(f"- {suggestion}")
</VSCode.Cell>
<VSCode.Cell id="u3v4w5x6" language="markdown">
## 7. Conclusion

In this notebook, we've built a practical mortgage rate estimator that combines traditional machine learning and LLM-based approaches. We've demonstrated:

1. Creating a synthetic mortgage dataset for model training
2. Building a Random Forest regression model as a baseline
3. Implementing an LLM-based estimator that provides detailed explanations
4. Creating a Flask API that serves both models
5. Building a simple web interface for interacting with the API
6. Comparing the outputs and insights from both approaches

**Key Takeaways:**

- **Machine Learning Approach:**
  - More consistent and systematic in its rate determination
  - Faster computation time
  - Limited to patterns present in training data
  - Lacks detailed explanations for its estimates

- **LLM-Based Approach:**
  - Provides rich explanations and context for rate determinations
  - Can incorporate domain knowledge not present in training data
  - Can analyze risk factors and provide personalized recommendations
  - Takes longer to compute and costs more per inference
  - May have more variability in its estimates

The ideal approach for production would be a hybrid system that:
1. Uses the ML model for initial estimates and efficiency
2. Uses the LLM for explanation, edge cases, and personalized recommendations
3. Implements a feedback loop to improve both models over time

This mortgage rate estimator demonstrates how financial institutions can leverage both traditional machine learning and modern LLMs to create more powerful, transparent, and helpful financial tools.
</VSCode.Cell>
</VSCode>```